# Doing Normalization and Setting up The batch size. 

In [3]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Define normalization transformations
val_transform = transforms.Compose([
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
])

# Load datasets with normalization (no augmentation)
train_dataset = ImageFolder("processed_data/train", transform=val_transform)
val_dataset = ImageFolder("processed_data/val", transform=val_transform)
test_dataset = ImageFolder("processed_data/test", transform=val_transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# Verify dataset sizes
print(f"Training dataset size: {len(train_dataset)} images")
print(f"Validation dataset size: {len(val_dataset)} images")
print(f"Test dataset size: {len(test_dataset)} images")


Training dataset size: 4800 images
Validation dataset size: 600 images
Test dataset size: 600 images


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import timm
import os


# Define the bare EfficientNet-B0 model

In [28]:

class BareEfficientNetB0(nn.Module):
    def __init__(self, num_classes=4):
        super().__init__()
        self.model = timm.create_model("efficientnet_b0", pretrained=True, num_classes=num_classes)
    
    def forward(self, x):
        return self.model(x)


In [30]:
# Training setup model 1
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bare2 = BareEfficientNetB0(num_classes=4).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(bare2.parameters(), lr=0.001)

print("DONE")


DONE


In [33]:
# model 2
print("Starting")
num_epochs = 20
for epoch in range(num_epochs):
    bare2.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = bare2(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

    # Validation
    bare2.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = bare2(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Validation Accuracy: {accuracy:.2f}%")

Starting
Epoch 1, Loss: 0.07188148938447739
Validation Accuracy: 99.67%
Epoch 2, Loss: 0.11954412172675348
Validation Accuracy: 99.67%
Epoch 3, Loss: 0.09582909294365284
Validation Accuracy: 99.67%
Epoch 4, Loss: 0.026464333012966867
Validation Accuracy: 100.00%
Epoch 5, Loss: 0.00844317286483753
Validation Accuracy: 100.00%
Epoch 6, Loss: 0.014769503286235401
Validation Accuracy: 99.83%
Epoch 7, Loss: 0.01976312504391558
Validation Accuracy: 99.83%
Epoch 8, Loss: 0.019273661120255525
Validation Accuracy: 100.00%
Epoch 9, Loss: 0.07283284577810263
Validation Accuracy: 99.50%
Epoch 10, Loss: 0.10914428449061234
Validation Accuracy: 99.83%
Epoch 11, Loss: 0.016352600168453742
Validation Accuracy: 100.00%
Epoch 12, Loss: 0.01425813367928034
Validation Accuracy: 99.83%
Epoch 13, Loss: 0.02178748345256281
Validation Accuracy: 100.00%
Epoch 14, Loss: 0.0065334327878463225
Validation Accuracy: 99.67%
Epoch 15, Loss: 0.005221256741663941
Validation Accuracy: 100.00%
Epoch 16, Loss: 0.006534091

In [35]:
# Save Bare EfficientNet-B0 model
torch.save(bare2.state_dict(), 'efficientnet_b0_bare_2.pth')
print("Model saved as 'efficientnet_b0_bare_2.pth'")

Model saved as 'efficientnet_b0_bare_2.pth'


In [36]:
import torch

# Assuming `model` is your trained or defined model
num_params = sum(p.numel() for p in bare2.parameters())
print(f"Total number of parameters: {num_params}")


Total number of parameters: 4012672


In [37]:
pip install torchsummary

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [38]:
from torchsummary import summary

# Assuming `model` is your trained or defined model
summary(bare2, (3, 224, 224))  # (3, 224, 224) for a 224x224 RGB image


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
          Identity-2         [-1, 32, 112, 112]               0
              SiLU-3         [-1, 32, 112, 112]               0
    BatchNormAct2d-4         [-1, 32, 112, 112]              64
            Conv2d-5         [-1, 32, 112, 112]             288
          Identity-6         [-1, 32, 112, 112]               0
              SiLU-7         [-1, 32, 112, 112]               0
    BatchNormAct2d-8         [-1, 32, 112, 112]              64
          Identity-9         [-1, 32, 112, 112]               0
           Conv2d-10              [-1, 8, 1, 1]             264
             SiLU-11              [-1, 8, 1, 1]               0
           Conv2d-12             [-1, 32, 1, 1]             288
          Sigmoid-13             [-1, 32, 1, 1]               0
    SqueezeExcite-14         [-1, 32, 1